In [1]:
# import dependencies

import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import psycopg2

import datetime as dt

from config import db_password



In [2]:
# load 

loadfile = "data/cdc_api_df_all.csv"
cdc_df = pd.read_csv(loadfile)

cdc_df.head()

,Unnamed: 0,Case_Month,State,ST_Code,County,Zip,Age_Range,Sex,Race,Ethnicity,Case_Positive_Specimen,Process,Exposure_yn,Current_Status,Symptom_Status,Hosp_yn,ICU_yn,Death_yn
0,0,2021-12,OH,39,HAMILTON,39061.0,18 to 49 years,Female,Black,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Unknown
1,1,2020-04,PA,42,LANCASTER,42071.0,18 to 49 years,Male,Asian,Non-Hispanic/Latino,1,Missing,Missing,Laboratory-confirmed case,Symptomatic,Yes,No,NaN
2,2,2020-12,UT,49,NaN,NaN,50 to 64 years,Male,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Unknown
3,3,2021-08,UT,49,NaN,NaN,65+ years,Male,White,Non-Hispanic/Latino,0,Missing,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,Unknown
4,4,2022-05,UT,49,NaN,NaN,65+ years,Male,NaN,NaN,0,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,NaN


In [3]:
# quick correction before I forget ;-)

cdc_df = cdc_df.rename(columns={'Zip': 'FIPS'})
cdc_df.head()

,Unnamed: 0,Case_Month,State,ST_Code,County,FIPS,Age_Range,Sex,Race,Ethnicity,Case_Positive_Specimen,Process,Exposure_yn,Current_Status,Symptom_Status,Hosp_yn,ICU_yn,Death_yn
0,0,2021-12,OH,39,HAMILTON,39061.0,18 to 49 years,Female,Black,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Unknown
1,1,2020-04,PA,42,LANCASTER,42071.0,18 to 49 years,Male,Asian,Non-Hispanic/Latino,1,Missing,Missing,Laboratory-confirmed case,Symptomatic,Yes,No,NaN
2,2,2020-12,UT,49,NaN,NaN,50 to 64 years,Male,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,Unknown
3,3,2021-08,UT,49,NaN,NaN,65+ years,Male,White,Non-Hispanic/Latino,0,Missing,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,Unknown
4,4,2022-05,UT,49,NaN,NaN,65+ years,Male,NaN,NaN,0,Missing,Missing,Laboratory-confirmed case,Missing,No,Missing,NaN


In [4]:
cdc_df.describe()

,Unnamed: 0,ST_Code,FIPS,Case_Positive_Specimen
count,111945.000000,111945.000000,107987.000000,111945.000000
mean,55972.000000,31.408763,31627.488966,0.163303
std,32315.882279,12.267760,11981.489326,2.306128
min,0.000000,1.000000,2020.000000,-118.000000
25%,27986.000000,21.000000,22033.000000,0.000000
50%,55972.000000,34.000000,34025.000000,0.000000
75%,83958.000000,39.000000,39155.000000,0.000000
max,111944.000000,78.000000,56041.000000,118.000000


In [5]:
cdc_df.isnull().sum()

Unnamed: 0                    0
Case_Month                    0
State                         0
ST_Code                       0
County                     3958
FIPS                       3958
Age_Range                   616
Sex                        2337
Race                      14330
Ethnicity                 15965
Case_Positive_Specimen        0
Process                       0
Exposure_yn                   0
Current_Status                0
Symptom_Status                0
Hosp_yn                       0
ICU_yn                        0
Death_yn                   2936
dtype: int64

In [6]:
#dropping null

cdc_df = cdc_df.dropna(how='any')

cdc_df.head()

,Unnamed: 0,Case_Month,State,ST_Code,County,FIPS,Age_Range,Sex,Race,Ethnicity,Case_Positive_Specimen,Process,Exposure_yn,Current_Status,Symptom_Status,Hosp_yn,ICU_yn,Death_yn
0,0,2021-12,OH,39,HAMILTON,39061.0,18 to 49 years,Female,Black,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Missing,Missing,Missing,Unknown
8,8,2022-01,TN,47,HAMILTON,47065.0,50 to 64 years,Female,Black,Non-Hispanic/Latino,0,Missing,Missing,Probable Case,Symptomatic,Missing,Missing,Missing
14,14,2020-08,OR,41,MARION,41047.0,18 to 49 years,Male,Unknown,Unknown,0,Laboratory reported,Yes,Laboratory-confirmed case,Symptomatic,No,Missing,No
15,15,2022-06,PA,42,DAUPHIN,42043.0,18 to 49 years,Male,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Unknown,Unknown,Unknown
16,16,2020-11,PA,42,BUCKS,42017.0,50 to 64 years,Female,White,Non-Hispanic/Latino,0,Missing,Missing,Laboratory-confirmed case,Symptomatic,Unknown,Unknown,Unknown


In [7]:
cdc_df.describe()

,Unnamed: 0,ST_Code,FIPS,Case_Positive_Specimen
count,90754.000000,90754.000000,90754.000000,90754.000000
mean,56375.109306,31.771206,31841.853770,0.155685
std,32444.487434,12.046259,12041.701866,2.039270
min,0.000000,2.000000,2020.000000,-65.000000
25%,28244.250000,22.000000,22033.000000,0.000000
50%,55757.500000,34.000000,34029.000000,0.000000
75%,85392.750000,41.000000,41051.000000,0.000000
max,111943.000000,56.000000,56041.000000,118.000000


In [8]:
cdc_df['Process'].value_counts()
# super high missing etc. rate -- consider removing column. 

Missing                            84060
Clinical evaluation                 2834
Laboratory reported                 1469
Routine surveillance                1269
Multiple                             489
Unknown                              254
Contact tracing of case patient      184
Provider reported                    132
Other                                 63
Name: Process, dtype: int64

In [9]:
cdc_df['Exposure_yn'].value_counts()
# here too, let's remove these and see what we get

Missing    79931
Yes         7660
Unknown     3163
Name: Exposure_yn, dtype: int64

In [10]:
# Removing PROCESS and EXPOSURE_YN columns due to high occurrence of 'Missing' values
# keeping them in previously had left only 735 remaining after dropping all rows w/ 'Missing' values

cdc_df.drop(['Process','Exposure_yn'], axis=1, inplace=True)

In [11]:
# dropping null variations

null_values = ("Missing", "Unknown", "NA")
columns = ["Case_Month", "State", "ST_Code", "County", "FIPS", "Age_Range", "Sex", "Race", "Ethnicity", "Case_Positive_Specimen",
          "Current_Status", "Symptom_Status", "Hosp_yn", "ICU_yn", "Death_yn"]

# ^ grouping all null values and then columns as variables and then trying to drop all indexed rows of all columns containing
#   all null values didn't want to take -- I'm assuming it's a me issue lol

# the below approach works fine but must be so inefficient 

to_drop = cdc_df[
    (cdc_df['Case_Month'] == "Missing") |
    (cdc_df['State'] == "Missing") |
    (cdc_df['ST_Code'] == "Missing") |
    (cdc_df['County'] == "Missing") |
    (cdc_df['FIPS'] == "Missing") |
    (cdc_df['Age_Range'] == "Missing") |
    (cdc_df['Sex'] == "Missing") |
    (cdc_df['Race'] == "Missing") |
    (cdc_df['Ethnicity'] == "Missing") |
    (cdc_df['Case_Positive_Specimen'] == "Missing") |
    (cdc_df['Current_Status'] == "Missing") |
    (cdc_df['Symptom_Status'] == "Missing") |
    (cdc_df['Hosp_yn'] == "Missing") |
    (cdc_df['ICU_yn'] == "Missing") |
    (cdc_df['Death_yn'] == "Missing") |

    (cdc_df['Case_Month'] == "Unknown") |
    (cdc_df['State'] == "Unknown") |
    (cdc_df['ST_Code'] == "Unknown") |
    (cdc_df['County'] == "Unknown") |
    (cdc_df['FIPS'] == "Unknown") |
    (cdc_df['Age_Range'] == "Unknown") |
    (cdc_df['Sex'] == "Unknown") |
    (cdc_df['Race'] == "Unknown") |
    (cdc_df['Ethnicity'] == "Unknown") |
    (cdc_df['Case_Positive_Specimen'] == "Unknown") |
    (cdc_df['Current_Status'] == "Unknown") |
    (cdc_df['Symptom_Status'] == "Unknown") |
    (cdc_df['Hosp_yn'] == "Unknown") |
    (cdc_df['ICU_yn'] == "Unknown") |
    (cdc_df['Death_yn'] == "Unknown") |

    (cdc_df['Case_Month'] == "NA") |
    (cdc_df['State'] == "NA") |
    (cdc_df['ST_Code'] == "NA") |
    (cdc_df['County'] == "NA") |
    (cdc_df['FIPS'] == "NA") |
    (cdc_df['Age_Range'] == "NA") |
    (cdc_df['Sex'] == "NA") |
    (cdc_df['Race'] == "NA") |
    (cdc_df['Ethnicity'] == "NA") |
    (cdc_df['Case_Positive_Specimen'] == "NA") |
    (cdc_df['Current_Status'] == "NA") |
    (cdc_df['Symptom_Status'] == "NA") |
    (cdc_df['Hosp_yn'] == "NA") |
    (cdc_df['ICU_yn'] == "NA") |
    (cdc_df['Death_yn'] == "NA")     
].index



cdc_df.drop(to_drop, inplace=True)
cdc_df.describe()


# and great! Removing PROCESS & EXPOSURE_YN columns + all 'Missing' now leaves us with 2875 records

,Unnamed: 0,ST_Code,FIPS,Case_Positive_Specimen
count,2875.000000,2875.000000,2875.000000,2875.000000
mean,57067.368348,38.770783,38837.337391,0.232000
std,32511.166579,10.422327,10401.513621,2.356106
min,128.000000,5.000000,5007.000000,-52.000000
25%,29425.500000,32.000000,32003.000000,0.000000
50%,56239.000000,42.000000,42003.000000,0.000000
75%,85661.500000,47.000000,47159.000000,0.000000
max,111934.000000,56.000000,56039.000000,30.000000


In [12]:
cdc_df.dtypes

Unnamed: 0                  int64
Case_Month                 object
State                      object
ST_Code                     int64
County                     object
FIPS                      float64
Age_Range                  object
Sex                        object
Race                       object
Ethnicity                  object
Case_Positive_Specimen      int64
Current_Status             object
Symptom_Status             object
Hosp_yn                    object
ICU_yn                     object
Death_yn                   object
dtype: object

In [13]:
#fixing my bools - originally thought .astype() would do it but that just converted everything into TRUE

cdc_df['Hosp_yn'] = cdc_df['Hosp_yn'].map({"No": False, "Yes": True})
cdc_df['ICU_yn'] = cdc_df['ICU_yn'].map({"No": False, "Yes": True})
cdc_df['Death_yn'] = cdc_df['Death_yn'].map({"No": False, "Yes": True})

cdc_df['Hosp_yn'].value_counts()

False    2469
True      406
Name: Hosp_yn, dtype: int64

In [14]:
# update columns & data types


cdc_df = cdc_df.rename(columns={'Case_Month': 'Date_Onset'})
cdc_df['Date_Onset'] = pd.to_datetime(cdc_df['Date_Onset'])

cdc_df['ST_Code'] = cdc_df['ST_Code'].astype(dtype='object')
cdc_df['FIPS'] = cdc_df['FIPS'].astype(dtype='object')
cdc_df['Age_Range'] = cdc_df['Age_Range'].astype(dtype='category')
cdc_df['Current_Status'] = cdc_df['Current_Status'].astype(dtype='category')
cdc_df['Symptom_Status'] = cdc_df['Symptom_Status'].astype(dtype='category')

cdc_df.dtypes

Unnamed: 0                         int64
Date_Onset                datetime64[ns]
State                             object
ST_Code                           object
County                            object
FIPS                              object
Age_Range                       category
Sex                               object
Race                              object
Ethnicity                         object
Case_Positive_Specimen             int64
Current_Status                  category
Symptom_Status                  category
Hosp_yn                             bool
ICU_yn                              bool
Death_yn                            bool
dtype: object

In [15]:
# month & year

cdc_df['Date_Onset_Month'] = cdc_df['Date_Onset'].dt.strftime('%m')
cdc_df['Date_Onset_Year'] = cdc_df['Date_Onset'].dt.strftime('%Y')

In [16]:
# creating sample proofing file for team

sample_clean = cdc_df.sample(50)
output = 'sample_post_clean.csv'
sample_clean.to_csv(output)

In [17]:
cdc_df.dtypes

Unnamed: 0                         int64
Date_Onset                datetime64[ns]
State                             object
ST_Code                           object
County                            object
FIPS                              object
Age_Range                       category
Sex                               object
Race                              object
Ethnicity                         object
Case_Positive_Specimen             int64
Current_Status                  category
Symptom_Status                  category
Hosp_yn                             bool
ICU_yn                              bool
Death_yn                            bool
Date_Onset_Month                  object
Date_Onset_Year                   object
dtype: object